In [2]:
from aocd import get_data

puzzle_input = get_data(day=12, year=2023)

In [3]:
it1 = """
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
""".strip()

Partie 1

In [4]:
def parse_input(input: str):
    records = []
    for row in input.split("\n"):
        record, group_counts = row.split(" ")
        records.append((record, tuple(int(i) for i in group_counts.split(","))))
    return records

parse_input(it1)

[('???.###', (1, 1, 3)),
 ('.??..??...?##.', (1, 1, 3)),
 ('?#?#?#?#?#?#?#?', (1, 3, 1, 6)),
 ('????.#...#...', (4, 1, 1)),
 ('????.######..#####.', (1, 6, 5)),
 ('?###????????', (3, 2, 1))]

In [4]:
def naive_count_matchs(record: str, group_counts: tuple[int]) -> int:
    nb_matchs = 0

    completes = []
    no_completes = [record]
    while no_completes:
        current:str = no_completes.pop()
        try:
            ind = current.index("?")
            no_completes.append(f"{current[:ind]}.{current[ind+1:]}")
            no_completes.append(f"{current[:ind]}#{current[ind+1:]}")
        except ValueError:
            completes.append(current)

    for comp in completes:
        groups_lengths = tuple(len(g) for g in comp.split(".") if g)
        nb_matchs += groups_lengths == group_counts

    return nb_matchs


def naive_count_arrangements(records):
    return sum([naive_count_matchs(record, group_counts) for record, group_counts in records])

naive_count_arrangements(parse_input(it1))

21

In [16]:
naive_count_arrangements(parse_input(puzzle_input))

7718

Amélioration Partie 1

In [5]:
def get_groups(record):
    return [g for g in record.split(".") if g]

In [6]:
record, count_groups = '.???..??...?##.', (1, 1, 3)

In [7]:
from functools import cache


@cache
def count_record(
    record: str,
    target: tuple,
    cur_groups: tuple = (),
    cur_count: int = 0,
):
    count = 0

    for i, c in enumerate(record):
        if c == "?":
            # On remplace le caractère courant par un dièse.
            # Dans ce cas on appelle récursivement la fonction avec un compteur de
            # dièse incrémenté de 1
            count += count_record(record[i + 1 :], target, cur_groups, cur_count + 1)

            # On remplace le caractère courant par un point dans ce cas deux cas de
            # figure :
            # - il y a un un groupe en courant, auquel cas on essaye de l'ajouter puis
            #   on continue
            # - il n'y a pas de groupe courant, auquel cas on continue
            if cur_count:
                # Si on peut encore ajouter des groupes, on l'ajoute puis on continue
                if (
                    len(cur_groups) < len(target)
                    and target[len(cur_groups)] == cur_count
                ):
                    cur_groups += (cur_count,)
                    cur_count = 0
                    continue

                # Sinon on arrête la boucle ici
                else:
                    break

        elif c == "#":
            cur_count += 1
        elif c == "." and cur_count:
            cur_groups += (cur_count,)
            cur_count = 0

    if cur_count:
        cur_groups += (cur_count,)

    if cur_groups == target:
        count += 1

    return count


for record, target in [
    ("???.###", (1, 1, 3)),
    (".??..??...?##.", (1, 1, 3)),
    ("?#?#?#?#?#?#?#?", (1, 3, 1, 6)),
    ("????.#...#...", (4, 1, 1)),
    ("????.######..#####.", (1, 6, 5)),
    ("?###????????", (3, 2, 1)),
]:
    print(count_record(record, target))

1
4
1
1
4
10


In [24]:
from functools import cache

@cache
def count_record(
    record: str,
    target: tuple,
    cur_count: int = 0,
) -> int:
    # S'il n'y a plus de groupe à former, on renvoie 1 s'il n'y a plus de #
    # (s'il y a des ? il faudrait de toute façon les prendre égal à .)
    if not target:
        return 0 if "#" in record else 1

    # S'il n'y a plus de record, on renvoie 1 sur le compteur de diese courant est
    # égal au dernier group à former
    if not record:
        return 1 if (cur_count,) == target else 0

    # S'il n'y a plus de ?, on compte directement les groupes
    if "?" not in record:
        groups = [len(g) for g in record.split(".") if g]

        if cur_count:
            if record[0] == "#":
                groups[0] += cur_count
                groups = tuple(groups)
            else:
                groups = (cur_count, *groups)
        else:
            groups = tuple(groups)

        return 1 if target == groups else 0

    # Sinon on procède récursivement
    c = record[0]

    if c == "#":
        return count_record(record[1:], target, cur_count + 1)
    elif c == ".":
        if cur_count > 0:
            if cur_count == target[0]:
                return count_record(record[1:], target[1:], 0)
            else:
                return 0
        else:
            return count_record(record[1:], target, 0)
    else:
        nb_diese = count_record(record[1:], target, cur_count + 1)

        if cur_count > 0:
            if cur_count == target[0]:
                return nb_diese + count_record(record[1:], target[1:], 0)
            else:
                return nb_diese

        return nb_diese + count_record(record[1:], target, 0)

# count_record("????.######..#####.", (1, 6, 5))

for record, target in [
    ("???.###", (1, 1, 3)),
    (".??..??...?##.", (1, 1, 3)),
    ("?#?#?#?#?#?#?#?", (1, 3, 1, 6)),
    ("????.#...#...", (4, 1, 1)),
    ("????.######..#####.", (1, 6, 5)),
    ("?###????????", (3, 2, 1)),
]:
    print(count_record(record, target))

1
4
1
1
4
10


In [25]:
def count_arrangements(records):
    return sum([count_record(record, group_counts) for record, group_counts in records])

count_arrangements(parse_input(it1))

21

In [26]:
count_arrangements(parse_input(puzzle_input))

7718

Partie 2

In [27]:
def dupliquer(records):
    return [
        ("?".join([record]*5), groups_counts*5) for record, groups_counts in records
    ]

it_records = dupliquer(parse_input(it1))
it_records

[('???.###????.###????.###????.###????.###',
  (1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3)),
 ('.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.',
  (1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3)),
 ('?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#?',
  (1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6)),
 ('????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...',
  (4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1)),
 ('????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.',
  (1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5)),
 ('?###??????????###??????????###??????????###??????????###????????',
  (3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1))]

In [28]:
count_arrangements(it_records)

525152

In [29]:
puzzle_records = dupliquer(parse_input(puzzle_input))
count_arrangements(puzzle_records)

128741994134728